In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d999a1b3b4f0c0becdacf0a3ccd4ee167ac5f32d354498e43c7edfa8d2257a1d
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Assignment3").setMaster("local[*]")
sc = SparkContext(conf=conf)



**Question 1:**



In [3]:
rdd = sc.textFile("InputQ1.txt")

rdd = rdd.map(lambda x: x.lower())

rdd1 = rdd.flatMap(lambda line: line.split(" "))

n = rdd1.count()

rdd_list = rdd1.collect()

consecutive_words = []

for i in range(n-2):
  if len(rdd_list[i]) > 4 and len(rdd_list[i+1]) > 4 and len(rdd_list[i+2]) > 4 and rdd_list[i][0] == rdd_list[i+1][0] == rdd_list[i+2][0]:
    consecutive_words.append((rdd_list[i], rdd_list[i+1], rdd_list[i+2]))

print("Part A:")
ConsecutiveWordsRdd = sc.parallelize(consecutive_words)
total_count = ConsecutiveWordsRdd.count()
print("Total count:", total_count)

counts_by_alphabet = ConsecutiveWordsRdd.map(lambda x: (x[0][0], 1))
counts_by_alphabet = counts_by_alphabet.reduceByKey(lambda x, y: x + y)

print("\n\nPart B:")
print("Counts by each alphabet:")
for alphabet, count in counts_by_alphabet.collect():
    print(alphabet, "=>", count)

combination_counts = ConsecutiveWordsRdd.map(lambda x: (x, 1))
combination_counts = combination_counts.reduceByKey(lambda x, y: x + y)

print("\n\nPart C:")
print("Combination counts:")
for combination, count in combination_counts.collect():
    print(combination, "=>", count)

Part A:
Total count: 6


Part B:
Counts by each alphabet:
h => 3
s => 1
m => 1
f => 1


Part C:
Combination counts:
('massive', 'murree', 'mountains.') => 1
('silly', 'stupid', 'samuel’s') => 1
('horrid', 'henry’s', 'hound') => 2
('henry’s', 'hound', 'hunts') => 1
('fantastic', 'fanciful', 'foursome.') => 1


**Question 2:**

In [4]:
# this function will calculate the simple matching coefficient(SMC) and will return the result
def smc(rdd1, rdd2):
    return sum([1 for i in range(len(rdd1)) if rdd1[i] == rdd2[i]]) / len(rdd1)

In [5]:
Input1 = sc.textFile("Input.txt")

Input2 = sc.textFile("Reference.txt")

Inputrdd = Input1.map(lambda line: line.split(":")).map(lambda x: (x[0], x[1].split()))

Refrdd = Input2.map(lambda line: line.split(":")).map(lambda x: (x[0], x[1].split()))

PossiblePairRdd = Inputrdd.cartesian(Refrdd)

smcRdd = PossiblePairRdd.map(lambda x: (x[0][0], (x[1][0], smc(x[0][1], x[1][1]))))

closestRefference = smcRdd.reduceByKey(lambda x, y: x if x[1] >= y[1] else y).sortBy(lambda x: x[0])

print("Patient ID \t Closest Reference \t SMC")
for (PatientID, (RefID, SMCValue)) in closestRefference.collect():
    print(PatientID, "\t\t", RefID, "\t\t\t", SMCValue)

Patient ID 	 Closest Reference 	 SMC
1 		 R1 			 0.8181818181818182
2 		 R1 			 0.9090909090909091
3 		 R1 			 0.8181818181818182
4 		 R2 			 0.9090909090909091


**Question 3:**

In [17]:
input_file = sc.textFile("InputQ3.txt")

authorsrdd = input_file.map(lambda line: tuple(line.split(" -> ")))

coauthorspair = authorsrdd.flatMap(lambda x: [((x[0], i), x[1].split(", ")) for i in x[1].split(", ")])

rdd = coauthorspair.map(lambda x: x)
authors = []
rddList = rdd.collect()
n=len(rddList)
for i in range(n):
  for j in range(i, n-1):
    rdd1 = sc.parallelize(rddList[i][0])
    rdd2 = sc.parallelize(rddList[j+1][0])
    intersection_rdd = rdd1.intersection(rdd2)
    if (intersection_rdd.collect() == rdd1.collect()):
      rdd3 = sc.parallelize(rddList[i][1])
      rdd4 = sc.parallelize(rddList[j+1][1])
      rdd5 = rdd3.intersection(rdd4)

      authors.append((rddList[i][0], rdd5.collect()))

rdd1 = sc.parallelize(authors)
rdd1 = rdd1.map(lambda x: (x[0], len(x[1]), x[1]))
rdd1.collect()

[(('Y. Lu', 'B. Cao'), 3, ['F. Glover', 'C. Rego', 'K. Kiani']),
 (('Y. Lu', 'C. Rego'), 1, ['B. Cao']),
 (('Y. Lu', 'F. Glover'), 1, ['B. Cao']),
 (('Y. Lu', 'K. Kiani'), 1, ['B. Cao']),
 (('B. Cao', 'C. Rego'), 1, ['Y. Lu']),
 (('B. Cao', 'K. Kiani'), 1, ['Y. Lu']),
 (('B. Hosseini', 'K. Kiani'), 0, [])]